#### Paquetes necesarios

In [1]:
import cv2  
import math 
from ultralytics import YOLO

#### Tarea 1: Reconocimiento de matrículas básico

In [2]:
# Carga del modelo
model = YOLO('yolov8n.pt')

# Guardamos imagen en variable
coche = cv2.imread("coche.jpg")

# Nombre de las distintas clases
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]


results = model("coche.jpg")

# Para cada detección
for r in results:
    boxes = r.boxes

    print(boxes)

    for box in boxes:
        # Contenedor
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
        
        # Confianza
        confidence = math.ceil((box.conf[0]*100))/100
        print("Confidence --->",confidence)

        # Clase
        cls = int(box.cls[0])
        print("Class name -->", classNames[cls])

        # Convierte identificador numérico de clase a un color RGB
        escala = int((cls / len(classNames)) * 255 * 3)
        if escala >= 255*2:
            R = 255
            G = 255
            B = escala - 255*2
        else:
            if escala >= 255:
                R = 255
                G = escala - 255
                B = 0
            else:
                R = escala
                G = 0
                B = 0

        # Dibuja el contenedor y clase
        cv2.rectangle(coche, (x1, y1), (x2, y2), (R, G, B), 3)
        cv2.putText(coche, classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

        # Muestra fotograma
        cv2.imshow('Vid', coche)

        cv2.waitKey(-1)

        # Destruye ventanas
        cv2.destroyAllWindows()


image 1/1 c:\Users\Usuario\Documents\ULPGC 2023-2024\VC\Repositorio GitHub\VC\Prctica 5\coche.jpg: 384x640 1 car, 149.0ms
Speed: 10.1ms preprocess, 149.0ms inference, 10.7ms postprocess per image at shape (1, 3, 384, 640)


ultralytics.engine.results.Boxes object with attributes:

cls: tensor([2.])
conf: tensor([0.9362])
data: tensor([[133.9155,  98.7597, 848.4449, 473.0604,   0.9362,   2.0000]])
id: None
is_track: False
orig_shape: (557, 990)
shape: torch.Size([1, 6])
xywh: tensor([[491.1802, 285.9101, 714.5294, 374.3007]])
xywhn: tensor([[0.4961, 0.5133, 0.7217, 0.6720]])
xyxy: tensor([[133.9155,  98.7597, 848.4449, 473.0604]])
xyxyn: tensor([[0.1353, 0.1773, 0.8570, 0.8493]])
Confidence ---> 0.94
Class name --> car


In [71]:
import pytesseract
from pytesseract import Output 

# Previamente debes descargar los ejecutables
# Si la ruta de Tesseract no está en el PATH, ruta al ejecutable
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

model_car = YOLO('yolov8n.pt')

# Guardamos imagen en variable
coche = cv2.imread("coche.jpg")

# Detectar coches
results_cars = model_car(coche)

# Para cada detección de coche
for r_car in results_cars:
    boxes_car = r_car.boxes

    for box_car in boxes_car:
        # Coordenadas del cuadro del coche
        x1_car, y1_car, x2_car, y2_car = box_car.xyxy[0]

        # Definir la región inferior para buscar matrículas
        roi_bottom = coche[int((y1_car + y2_car) / 2):int(y2_car), int(x1_car):int(x2_car)]

        # Convertir la región a escala de grises
        gray_roi_bottom = cv2.cvtColor(roi_bottom, cv2.COLOR_BGR2GRAY)

        # Aplicar umbralización adaptativa
        _, thresh = cv2.threshold(gray_roi_bottom, 127, 255, cv2.THRESH_BINARY)

        # Buscar contornos en la región inferior
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Filtrar contornos
        for contour in contours:
            # Filtrar por área del contorno
            area = cv2.contourArea(contour)
            if 500 < area < 5000:  # Ajusta estos valores según tu caso

                # Aproximación de polígono para evaluar la forma
                epsilon = 0.02 * cv2.arcLength(contour, True)
                approx = cv2.approxPolyDP(contour, epsilon, True)

                # Filtrar por la relación de aspecto y la forma
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = float(w) / h
                if 2 < aspect_ratio < 5 and len(approx) == 4:
                    # Ajustar las coordenadas al espacio global de la imagen
                    x, y, w, h = int(x + x1_car), int(y + (y1_car + y2_car) / 2), int(w), int(h)

                    # Extraer la región de la matrícula como una nueva imagen
                    license_plate_region = coche[y:y + h, x+10:x + w]

                    cv2.imshow('Matrícula', license_plate_region)

                    # Aplicar pytesseract a la región de la matrícula
                    text = pytesseract.image_to_string(license_plate_region, config='--psm 8', output_type=Output.STRING)
                    print("Matrícula:", text)

                    # Dibujar cuadro alrededor del contorno
                    cv2.rectangle(coche, (x, y), (x + w, y + h), (0, 255, 0), 3)

# Muestra la imagen con las detecciones
cv2.imshow('Vid', coche)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 384x640 1 car, 109.6ms
Speed: 0.0ms preprocess, 109.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


Matrícula: 0478 LFR

